# Visualizing sensory landscapes

Sensory landscapes of diverse modalities are available :
- Olfactory (Odorscape)
- Temperature (Thermoscape)
- Wind (Windscape)

Initialize the larvaworld registry. This loads some components from disc and builds the rest on the fly.

In [ ]:
import numpy as np

from larvaworld.lib import aux, reg, process
from larvaworld.lib.sim.single_run import ExpRun
#reg.VERBOSE=1



In [ ]:
from larvaworld.lib.param.composition import Odor

def get_odorscape(m):
    if m == 'Diffusion':
        oS = reg.gen.DiffusionValueLayer(gaussian_sigma=(0.95, 0.5),evap_const=0.9)
        oR = Odor.oD(id='Odor_R')
        oL = Odor.oD(id='Odor_L')
    elif m == 'Gaussian':
        oS = reg.gen.GaussianValueLayer()
        oR = Odor.oG(id='Odor_R')
        oL = Odor.oG(id='Odor_L')
    else :
        raise ValueError ('Not implemented')

    kws = {
        'group':'Source',
        'radius':0.003,
        'amount':0.0,
    }
    
    sus = {
        **reg.gen.Food(unique_id = 'Source_L', c='blue', odor=oL, pos=(-0.01, 0.0), **kws).entry(),
        **reg.gen.Food(unique_id = 'Source_R', c='cyan', odor=oR, pos=(0.01, 0.0), **kws).entry(),
    }

    return aux.AttrDict({'food_params' : reg.gen.FoodConf(source_units=sus), 'odorscape' : oS})


def get_windscape(m):
    kws = {
        'duration':5,
        'speed':50,
    }

    Npuffs = 10
    if m == 'single':
        puffs = {
            i: reg.gen.AirPuff(direction=i / Npuffs * 2 * np.pi, start_time=5 + 10 * i, **kws).nestedConf
            for i in range(Npuffs)}
        ws = 0.0
    elif m == 'repetitive':
        puffs = {
            'puff_group': reg.gen.AirPuff(direction=np.pi / 4, start_time=5, N=Npuffs,
                                       interval=10.0, **kws).nestedConf}
        ws = 0.0
    elif m == 'no':
        puffs = {}
        ws = 10.0
    else:
        raise ValueError('Not implemented')

    wS=reg.gen.WindScape(wind_direction=0.0, wind_speed=ws, puffs=puffs)

    bl={'Border': reg.gen.Border(vertices=[(-0.03, 0.02), (0.03, 0.02)])}
    bl={}
    return aux.AttrDict({'windscape' : wS, 'border_list' : bl})


def get_env(Om=None, Pm=None):
    dO=get_odorscape(m=Om) if Om is not None else {}
    dW=get_windscape(m=Pm) if Pm is not None else {}
    return reg.gen.Env(**dO, **dW)

def get_conf(Om=None, Pm=None,Wm=None, duration=0.3, experiment='dish') : 
    if Om is not None and Pm is None:
        filename=f'{Om}_odorscape'
    elif Om is None and Pm is not None:
        filename=f'{Pm}_air-puffs_variable_wind_{Wm}'
    else :
        filename=f'{Om}_odorscape_{Pm}_air-puffs_variable_wind_{Wm}'
        
    p=reg.gen.Exp(env_params=get_env(Om=Om, Pm=Pm), duration=duration, experiment=experiment).nestedConf
    p.trials={}
    conf= aux.AttrDict({
        'parameters' : p,
    'screen_kws' : {'mode': 'video', 'fps' : 10,'show_display' : True,'save_video' : True,'media_dir' : './media', 'video_file' : filename,
                   'odor_aura' : True}
    })
    return conf
    
def run_scape(Om=None, Pm=None,Wm=None, **kwargs) :
    env = ExpRun(**get_conf(Om=Om,Pm=Pm,Wm=Wm, **kwargs))
    env.sim_setup(steps=env.p.steps, seed=None)
    if Om is not None:
        env.odor_layers['Odor_L'].visible = True
        #env.odor_layers['Odor_R'].visible = True
        #env.screen_manager.odor_aura = True
    if Pm is not None :
        env.windscape.visible = True
    while env.running:
        if Wm == 'direction':
            env.windscape.set_wind_direction((env.t / 10 / np.pi) % (2 * np.pi))
        elif Wm == 'speed':
            env.windscape.wind_speed = env.t % 100
        env.sim_step()
    env.end()

In [ ]:
#Run single scape

run_scape(Om='Diffusion', Pm='repetitive',Wm='speed', duration=2)

In [ ]:
#Run all scape combinations

#exp='Wind&Odorscape visualization'
#exp='dish'

for Om in ['Gaussian', 'Diffusion', None] :
    for Pm in [None,'single', 'repetitive', 'no'] :
        for Wm in [None,'direction', 'speed', 'no'] :
            if Pm is None and Wm is not None :
                continue
            elif Pm is not None and Wm is None :
                continue
            elif Om is None and Pm is None :
                continue
            run_scape(Om=Om,Pm=Pm,Wm=Wm)
            #print(conf.screen_kws.video_file)

In [ ]:
import numpy as np

from ...lib import reg, aux
from ...lib.sim.single_run import ExpRun

exp='Wind&Odorscape visualization'



def get_conf(odor_mode, puff_mode, wind_mode):
    media_name = f'{odor_mode}_{puff_mode}_air-puffs_variable_wind_{wind_mode}'

    if odor_mode == 'diffusion_odorscape':
        odorscape = reg.get_null('odorscape', odorscape='Diffusion', grid_dims=(41, 41), gaussian_sigma=(0.95, 0.95),
                                  evap_const=0.9)
        oR = reg.get_null('odor', id='Odor_R', intensity=300.0, spread=0.1)
        oL = reg.get_null('odor', id='Odor_L', intensity=300.0, spread=0.1)
    elif odor_mode == 'gaussian_odorscape':
        odorscape = reg.get_null('odorscape', odorscape='Gaussian')
        oR = reg.get_null('odor', id='Odor_R', intensity=2.0, spread=0.0002)
        oL = reg.get_null('odor', id='Odor_L', intensity=2.0, spread=0.0002)
    else:
        raise ValueError('Not implemented')
    sus = {
        'Source_L': reg.get_null('source', default_color='blue', group='Source', radius=0.003, amount=0.0, odor=oL,
                                  pos=(0.0, 0.0)),
    }

    Npuffs = 100
    if puff_mode=='single' :
        puffs = {i: reg.get_null('air_puff', duration=2, speed=40, direction=i / Npuffs * 2 * np.pi, start_time=5 + 10 * i)
                 for i in range(Npuffs)}
        wind_speed = 0.0
    elif puff_mode == 'repetitive':
        puffs = {'puff_group': reg.get_null('air_puff', duration=2, speed=40, direction=np.pi, start_time=5, N=Npuffs,
                                             interval=5.0)}
        wind_speed = 0.0
    elif puff_mode == 'no':
        puffs = {}
        wind_speed = 30.0
    else:
        raise ValueError('Not implemented')

    windscape = reg.get_null('windscape', wind_direction=-np.pi / 2, wind_speed=wind_speed, puffs=puffs)

    conf = {'parameters': aux.AttrDict({
        'sim_params': reg.get_null('sim_params', duration=2.0),
        'env_params': reg.get_null('Env',
                               arena=reg.get_null('arena', shape='rectangular', dims=(0.3, 0.3)),
                               food_params={'source_groups': {},
                                            'food_grid': None,
                                            'source_units': sus},
                               odorscape=odorscape,
                               windscape=windscape),
        'larva_groups': {},
        'trials': {},
        'collections': None,
        'experiment': exp,
    }),
        'screen_kws': {
            'vis_kwargs': reg.get_null('visualization', mode='video', video_speed=10, media_name=media_name)},
        'save_to': '.'
    }
    return conf



for odor_mode in ['gaussian_odorscape', 'diffusion_odorscape'] :
    for puff_mode in ['single', 'repetitive', 'no'] :
        for wind_mode in ['direction', 'speed', 'no'] :

            conf = get_conf(odor_mode, puff_mode, wind_mode)
            env = ExpRun(**conf)
            env.setup(**env._setup_kwargs)
            env.odor_layers['Odor_L'].visible = True
            env.odor_aura = True
            env.windscape.visible = True
            while env.t <= env.Nsteps:
                if wind_mode == 'direction':
                    env.windscape.set_wind_direction((env.t / 10 / np.pi) % (2 * np.pi))
                if wind_mode == 'speed':
                    env.windscape.wind_speed = env.t % 100
                env.t += 1
                env.step()
            env.end()
